## Homework 10_3

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### 10.11

p_value = $P\left(T \leq 2 \mid \theta = \frac{1}{2}\right) = P\left(H \geq 8 \mid \theta = \frac{1}{2} \right) = \sum_{i=8}^{10} \binom{10}{i}\left(\frac{1}{2}\right)^{i}\left(\frac{1}{2}\right)^{10-i} = \frac{1}{2^{10}}\left( \binom{10}{8} + \binom{10}{9} + \binom{10}{10} \right) = \frac{56}{2^{10}} \approx 0.0546875$.

If $\alpha = 0.1$, then it does justify rejecting the null hypothesis.  However, with $\alpha = 0.05$, it's not justified.

### 10.12

Since $\hat{\beta_{1}} = (X_{1}^{T}X_{1})^{-1}X_{1}^{T}Y$, with $Y = X_{1}\beta_{1} + X_{2}\beta_{2} + \eta$, we can caluclate $\mathbb{E}(\hat{\beta_{1}} \mid X_{1})$ as :
$
\begin{align*}
\mathbb{E}(\hat{\beta_{1}} \mid X_{1}) &= \mathbb{E}\left[(X_{1}^{T}X_{1})^{-1}\left(X_{1}\beta_{1}+X_{2}\beta_{2}+\eta\right) \mid X_{1}\right] \\
&= \mathbb{E}\left[ \beta_{1} + (X_{1}^{T}X_{1})^{-1}X_{1}^{T}X_{2}\beta_{2} + (X_{1}^{T}X_{1})^{-1}X_{1}^{T}\eta \mid X_{1} \right] \\
&= \mathbb{E}\left[ \beta_{1} \mid X_{1} \right] + \mathbb{E}\left[ (X_{1}^{T}X_{1})^{-1}X_{1}^{T}X_{2}\beta_{2} \mid X_{1} \right] + \mathbb{E}\left[ (X_{1}^{T}X_{1})^{-1}X_{1}^{T}\eta \mid X_{1} \right] \\
&= \beta_{1} + (X_{1}^{T}X_{1})^{-1}X_{1}^{T}\mathbb{E}\left[ X_{2}\beta_{2} \mid X_{1} \right] + (X_{1}^{T}X_{1})^{-1}X_{1}^{T}\mathbb{E}\left[ \eta \mid X_{1} \right] \\
&= \beta_{1} + (X_{1}^{T}X_{1})^{-1}X_{1}^{T}\mathbb{E}\left[ X_{2}\beta_{2} \mid X_{1} \right]
\end{align*}
$
so if $X_{2}\beta_{2} \not= 0$, then $\mathbb{E}(\hat{\beta_{1}} \mid X_{1}) \not= \beta_{1}$, and is therefore biassed.

### 10.13

In [2]:
df = pd.read_csv('wages.csv')

In [4]:
y = df['wage']
X1 = df['female']
X1 = sm.add_constant(X1)

In [6]:
results = sm.OLS(y,X1).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     68.54
Date:                Tue, 05 Dec 2017   Prob (F-statistic):           1.04e-15
Time:                        20:33:55   Log-Likelihood:                -1400.7
No. Observations:                 526   AIC:                             2805.
Df Residuals:                     524   BIC:                             2814.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0995      0.210     33.806      0.0

In [9]:
X = df.filter(['female','educ','exper','tenure','married'])
X = sm.add_constant(X)
new_res = sm.OLS(y,X).fit()
print(new_res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     60.61
Date:                Tue, 05 Dec 2017   Prob (F-statistic):           1.02e-49
Time:                        20:43:14   Log-Likelihood:                -1312.3
No. Observations:                 526   AIC:                             2637.
Df Residuals:                     520   BIC:                             2662.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6182      0.723     -2.238      0.0

The coefficient for female increased by about $0.75$, which would indicate that at least some of the correlation from before was influenced by these other variables as well.  Whether or not you're female seems to be the main factor as it's coefficent still has the greatest absolute value.  Years of education and whether or not you're married are the other major factors using these variables, although being married is just beyond the cut off with $\alpha=0.05$.

In [10]:
df['female & married'] = df['female']*df['married']
Y = df.filter(['female','educ','exper','tenure','married','female & married'])
Y = sm.add_constant(Y)
final_res = sm.OLS(y,Y).fit()
print(final_res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     55.84
Date:                Tue, 05 Dec 2017   Prob (F-statistic):           3.90e-53
Time:                        20:50:50   Log-Likelihood:                -1302.1
No. Observations:                 526   AIC:                             2618.
Df Residuals:                     519   BIC:                             2648.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.3942      0.730  

Almost the entirety of the constant for being strictly female went into be female and married.  The p-values would indicate that there isn't really a correlation to being female and earning less, but rather being a married female and earning less.

In [15]:
print(df.filter(['female & married']).astype(bool).sum())
print(df.filter(['female']).astype(bool).sum())

female & married    132
dtype: int64
female    252
dtype: int64
